In [24]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-135140")
exp = Experiment(workspace=ws, name="BankMarketing-comparison")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135140
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-135140


In [49]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pandas~=0.25.0
- scikit-learn
- numpy
- pip=20.2.4
- pip:
    - azurml-sdk==1.20.0
    - azureml-core==1.20.0
    - azurml-pipeline-core==1.20.0
    - azureml-telemetry==1.20.0
    - azureml-defaults==1.20.0
    - azureml-interpret==1.20.0
    - azureml-automl-core==1.20.0
    - azureml-train-automl-client==1.20.0
    - azureml-train-automl-runtime==1.20.0
    - inference-schema
    - py-cpuinfo==5.0.0
    - boto3==1.15.18
    - botocore==1.18.18

Writing conda_dependencies.yml


In [50]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

In [51]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = 'automl-hyperdrive-comp'

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [52]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
    'C': uniform(0.001, 1.0), # For regularization
    'max_iter': choice(10, 50, 100) #Max number of epochs
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
compute_target = ws.compute_targets[cpu_cluster_name]
est = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--C', '--max_iter', 1.0],
                      compute_target=compute_target,
                      run_config = RunConfiguration(),
                      environment=sklearn_env)

# est._compute_target = compute_target
# est._get_script_run_config = RunConfiguration()

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=100,
                                    max_concurrent_runs=10)

In [53]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)

from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

AttributeError: 'ScriptRunConfig' object has no attribute '_compute_target'

In [30]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id:', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Regularization Rate:', parameter_values[1])
print('Number of Iterations:', parameter_values[3])

best_run.download_file("/outputs/model.joblib", "hyperdrive_best_model.joblib")

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###